# Type Stable Assembly of Vectors and Matrices 
# Using Preallocation and Static Arrays 

Here we collect various ideas on the type-stable generation of matrices and vectors. 

Information in this notebook was discussed on Discourse at [this post](https://discourse.julialang.org/t/vector-of-vectors-non-allocating/109762/6). 

## Import Package

In [1]:
using LinearAlgebra # to obtain the cross product in later examples 
using StaticArrays
using BenchmarkTools

## Section 1: Generate  N-Vectors of Float64

<b> Lessons Learned </b>: 
1. <i>undef</i> means <i>uninitialized</i>.  
2. the keyword <i>missing</i> is explained at [Julia manual for missing](https://docs.julialang.org/en/v1/manual/missing/);

#### Working Version: Size of Vector Known at Compile Time 
This version is type-stable. This versions generates a static vector. This version might be inappropriate to generate very large vectors. 

In [23]:
function genvec1()
    length = 4 
    aaa = SVector{length, Float64}(ones(length)) 
    return aaa 
end

genvec1 (generic function with 1 method)

In [24]:
a = genvec1()

4-element SVector{4, Float64} with indices SOneTo(4):
 1.0
 1.0
 1.0
 1.0

In [25]:
@code_warntype genvec1()

MethodInstance for genvec1()
  from genvec1() in Main at In[23]:1
Arguments
  #self#::Core.Const(genvec1)
Locals
  aaa::SVector{4, Float64}
  length::Int64
Body::SVector{4, Float64}
1 ─      (length = 4)
│   %2 = Core.apply_type(Main.SVector, length::Core.Const(4), Main.Float64)::Core.Const(SVector{4, Float64})
│   %3 = Main.ones(length::Core.Const(4))::Vector{Float64}
│        (aaa = (%2)(%3))
└──      return aaa



#### Another Working Version: Generates a Non-static Vector

In [ ]:
function genvec2(N)
    result = Vector{Float64}(undef,N)
    for i=1:N
        result[i] = 2*i 
    end 
    return result
end 

In [26]:
N = 10;    @btime genvec2(N); 
N = 100;   @btime genvec2(N);
N = 1000;  @btime genvec2(N);

  745.629 ns (11 allocations: 752 bytes)
  834.507 ns (11 allocations: 2.17 KiB)
  1.333 μs (12 allocations: 16.33 KiB)


### Failing Version: Size of Vector Unknown at Compile Time

This version is *not* type-stable. The macro @btime generates text is red font.  

In [10]:
function genvec3(length::Int64)
    aaa = SVector{length, Float64}(ones(length)) # value of length unknown at compile time 
    return aaa 
end

genvec2 (generic function with 1 method)

In [11]:
genvec3(4)

4-element SVector{4, Float64} with indices SOneTo(4):
 1.0
 1.0
 1.0
 1.0

In [12]:
@code_warntype genvec3(4)

MethodInstance for genvec2(::Int64)
  from genvec2(length::Int64) in Main at In[10]:1
Arguments
  #self#::Core.Const(genvec2)
  length::Int64
Locals
  aaa::Any
Body::Any
1 ─ %1 = Core.apply_type(Main.SVector, length, Main.Float64)::Type{SVector{_A, Float64}} where _A
│   %2 = Main.ones(length)::Vector{Float64}
│        (aaa = (%1)(%2))
└──      return aaa



## Section 2: Generate N-Vector of 2-Tuple of Int64

Here we consider generating N-vectors of two-vectors on long integer.   

### Native Bad Solutions to Avoid  

In the solution that follows, a vector of size two is allocated for each value in the loop. The number that @btime increases proportionally with problem size. This solution provides a prototype we wish to avoid. 

In [27]:
# uses a Vector of Vector and thus need to allocate 1 outer Vector and N small Vectors
function genvectuple2(N)
    result = Vector{Vector{Int64}}(undef, N) # allocate one Vector
    for i = 1:N
        result[i] = [2i, 2i+1] # allocate one Vector per loop
    end 
    return result
end

genvectuple2 (generic function with 1 method)

In [ ]:
N = 10;    @btime genvectuple2(N); 
N = 100;   @btime genvectuple2(N);
N = 1000;  @btime genvectuple2(N);

In [30]:
# like genvectuple2 uses a Vector of Vector and thus need to allocate 1 outer Vector and N small Vectors
function genvectuple3(N)
    result = [zeros(Int64,2) for i=1:N] 
    # result = Vector{Vector{Int64}}(undef, N) # UndefRefError: access to undefined reference in line 5 
    for i=1:N
        # result[i] = (2*i, 2i+1) # using a generator - cannot convert Tuple{Int64, Int64} to Vector{Int64}  
        result[i] .= (2*i, 2i+1) # using a generator   
    end 
    return result
end 

genvectuple3 (generic function with 1 method)

In [29]:
N = 10;    @btime genvectuple3(N); 
N = 100;   @btime genvectuple3(N);
N = 1000;  @btime genvectuple3(N);

  244.975 ns (11 allocations: 928 bytes)
  2.009 μs (101 allocations: 8.69 KiB)
  19.625 μs (1001 allocations: 86.06 KiB)


In [31]:
# this implementation fails while it does work for both mikmore and abraemer 
# probably need to update version of Julia used 
# needs only 1 allocation because it then uses views into the memory. 
# however each view is an indirection and so it is not as fast as it could be
function genvectuple4(N)
    result = eachcol(zeros(Int64, 2, N)) # allocate one Matrix but read it like many Vectors
    for i = 1:N
        result[i] .= (2i, 2i+1) # fill the sliced matrix with values
    end 
    return result
end

genvectuple4 (generic function with 1 method)

### More Bad Solutions Even Though Static Arrays Are Used 

In [ ]:
# genvectuple5 is really the very same thing as genvectuple3. 
# In some sense a SVector is just a fancy tuple, so are genvec5 and genvec3 identical.
function genvectuple5(N)
    # result = Vector{Vector{Int64}}(undef, N)
    result = [zeros(Int64,2) for i=1:N]
    # result = eachcol(zeros(Int64, 2, N)) # allocate one Matrix but read it like many Vectors
    for i = 1:N
        result[i] .= SVector(2i, 2i+1) # fill the sliced matrix with values
    end 
    return result
end

In [32]:
# use as function 
# genvec6 does not work for me. 
# But conceptually it would use a SVector of SVectors which is theoretically fine
# but would cause a lot of trouble for the compiler as it now has to compile a new method 
# for every length of the outer vector. 
# That’s why StaticArrays.jl recommends not using it beyond ~100 elements or so.
function genvectuple6(N) 
    result = @SVector [ ( 2*i, 2*i+1 ) for i=1:N ]
    return result
end

genvectuple6 (generic function with 1 method)

### Finally a Good Solution - Hurray! 

The implementations genvec7 and genvec8 do the most sensible thing: They use a Vector of SVector. That is best of both worlds. We can have a varying amount of small vectors without troubling the compiler but operations on the small vectors is still fast because Julia knows their length. In fact Julia can optimize away even the allocation of the small vectors because they all have the same known size and are thus stored inline inside the outer array (this is not possible with Vector because a Vector can grow in size thus needs to live in the heap separately). Btw: This approach is conceptually very close to genvec4. The data of the inner vectors is packed together in a single large array in memory. The main difference is just the way of accessing it. Here SVector makes a large difference compared to views, because again views can have varying sizes and thus Julia cannot optimize them away and so there is an additional layer of indirection.

The solution that follows employs both static arrays and map. Can both of these ingredients be avoided? 

In [33]:
function genvec7(N)
    result = map(i -> @SVector[2i,2i+1], 1:N) # allocate one Vector
    return result
end

genvec7 (generic function with 1 method)

In [35]:
function genvec8(N)
    elemtype = typeof(@SVector[1,2])
    result = Vector{elemtype}(undef,N) # vector of SVector
    for i = eachindex(result)
        result[i] = @SVector[2i, 2i+1] # fill the slots (no broadcast!)
    end
    return result
end

genvec8 (generic function with 1 method)

In [36]:
N = 10;    @btime genvec8(N); 
N = 100;   @btime genvec8(N);
N = 1000;  @btime genvec8(N);

  31.617 ns (1 allocation: 224 bytes)
  77.246 ns (1 allocation: 1.77 KiB)
  630.737 ns (1 allocation: 15.75 KiB)


## Section 3: Constructing Matrices   

### Script Version for easy experimenting

In [ ]:
N = 3;
A = zeros(3*N, 3*N)
display(A)

### Function Version for benchmarking: Version 1: Using Generator and Static Arrays 

In [14]:
function genMat1(N)

    #..initizalize 
    A = zeros(3*N, 3*N)
    
    #..loop over element in the mesh 
    for i = 1:N 
       for j = 1:N 
          Aloc = SMatrix{3,3}(i+j for i=1:3, j=1:3)
          A[3*(i-1)+1:3*i,3*(j-1)+1:3*j] = Aloc 
       end
    end    
    
    return A 
end 

genMat1 (generic function with 1 method)

In [15]:
N = 10;    @btime genMat1(N); 
N = 100;   @btime genMat1(N); 
N = 1000;  @btime genMat1(N); 
N = 10000; @btime genMat1(N); 

  1.683 μs (1 allocation: 7.19 KiB)
  151.917 μs (2 allocations: 703.17 KiB)
  19.650 ms (2 allocations: 68.66 MiB)
  3.003 s (2 allocations: 6.71 GiB)


### Function Version 2: Using Preallocation and Explicit Double For Loop  

In [16]:
function genMat2(N)

    #..initizalize 
    A = zeros(3*N, 3*N)
    Aloc = zeros(3,3)
    
    #..loop over element in the mesh 
    for i = 1:N 
       for j = 1:N 
            
            for ii=1:3
                for jj=1:3
                    Aloc[ii,jj] = i+j 
                end 
            end 
            
          A[3*(i-1)+1:3*i,3*(j-1)+1:3*j] = Aloc 
       end
    end    
    
    return A 
end 

genMat2 (generic function with 1 method)

In [ ]:
N = 10;    @btime genMat2(N); 
N = 100;   @btime genMat2(N); 
#N = 1000;  @btime genMat2(N); 
#N = 10000; @btime genMat2(N); 

### Function Version 3: Fails: Using Preallocation Avoiding Explicit Double For Loop  

In [18]:
function genMat3(N)

    #..initizalize 
    A = zeros(3*N, 3*N)
    Aloc = zeros(3,3)
    
    #..loop over element in the mesh 
    for i = 1:N 
       for j = 1:N 
                        
          [Aloc[i,j] = i+j for i=1:3, j=1:3]
            
          A[3*(i-1)+1:3*i,3*(j-1)+1:3*j] = Aloc 
       end
    end    
    
    return A 
end 

genMat3 (generic function with 1 method)

In [19]:
N = 10;    @btime genMat3(N); 
N = 100;   @btime genMat3(N); 
#N = 1000;  @btime genMat2(N); 
#N = 10000; @btime genMat2(N);|

  4.101 μs (102 allocations: 19.81 KiB)
  393.459 μs (10003 allocations: 1.91 MiB)


### Function Version 4: Using Preallocation and Explicit Double For Loop: More Complex Example 

In [20]:
function myCross!(result,a,b)
# computes outer product of a and b and stores results oin w
# assumption is that memory for w has been allocated outside this function     
    result[1] = a[2]*b[3] - a[3]*b[2]
    result[2] = -a[1]*b[3] + a[3]*b[1]
    result[3] = a[1]*b[3] - a[3]*b[1]
    return result
end 

myCross! (generic function with 1 method)

In [21]:
function genMat4(N)

    #..initizalize 
    A = zeros(3*N, 3*N)
    Aloc = zeros(3,3)
    a = [0.;1;0]
    at = [0. 1 0]
    b = [0.;0;1]
    w = zeros(3)
    
    #..loop over element in the mesh 
    for i = 1:N 
       for j = 1:N 

            for ii=1:3
                for jj=1:3
                    w = myCross!(w,a,b)
                    Aloc[ii,jj] = norm(w) 
                end 
            end 
            
          A[3*(i-1)+1:3*i,3*(j-1)+1:3*j] = Aloc 
       end
    end    
    
    return A 
end

genMat4 (generic function with 1 method)

In [22]:
N = 10;    @btime genMat4(N); 
N = 100;   @btime genMat4(N); 
#N = 1000;  @btime genMat2(N); 
#N = 10000; @btime genMat2(N);

  10.667 μs (18 allocations: 7.95 KiB)
  1.028 ms (19 allocations: 703.94 KiB)
